# 3rd Level - Random Effects (RFX)

In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel

import numpy as np
import os
import shutil
from PIL import Image

In [2]:
# path initiallization
data = r"/home/anakin/Desktop/ds000113/output/2nd_level"
output = r"/home/anakin/Desktop/ds000113/output/3rd_level"
mask_file = "/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz"

# define parameters based on previous notebooks
num_of_contrasts = 15
num_of_subjects = 8
contrast_list = [str(i) for i in range(0,num_of_contrasts)]

In [3]:
infosource = Node(IdentityInterface(fields=["contrast_list"]), name="infosource")
infosource.iterables = [("contrast_list", contrast_list)]
templates = {"copes":     "*/fixed_effects/*/_fixed_eff_model{contrast_list}/cope1.nii.gz",
             "varcopes":  "*/fixed_effects/*/_fixed_eff_model{contrast_list}/varcope1.nii.gz"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data
selectfiles.inputs.contrast_list=1

substitutions = []
datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [6]:
def pick_first(out_stat):
    """
    this function takes the first arguement (number of voxels in mask file) from the output
    of num_of_voxels node to pass to cluster correction.
    """
    return int(out_stat[0])

In [7]:
# nodes initiallization
copemerge = Node(fsl.Merge(),name="copemerge")
copemerge.inputs.dimension = "t"

varcopemerge = Node(fsl.Merge(),name="varcopemerge")
varcopemerge.inputs.dimension = "t"

level2model = Node(fsl.L2Model(), name='l2model')
level2model.inputs.num_copes = num_of_subjects

random_eff_model = Node(fsl.FLAMEO(),name="random_eff_model")
random_eff_model.inputs.run_mode="ols"
random_eff_model.inputs.mask_file= mask_file

cluster_corr = Node(fsl.Cluster(),name="cluster_corr")
cluster_corr.inputs.threshold = 3.29                       # z threshold - 
cluster_corr.inputs.pthreshold = 0.001                     # p threshold
cluster_corr.inputs.use_mm = True
cluster_corr.inputs.out_threshold_file = True
cluster_corr.inputs.out_localmax_txt_file = True
cluster_corr.inputs.out_pval_file = True
cluster_corr.inputs.out_index_file = True

randomise = Node(fsl.Randomise(), name="randomise")
randomise.inputs.mask = mask_file
randomise.inputs.tfce = True
randomise.inputs.vox_p_values = True
randomise.inputs.num_perm = 1000

smooth_estimate = Node(fsl.SmoothEstimate(), name="smooth_estimate")
smooth_estimate.inputs.mask_file = mask_file 

num_of_voxels = Node(fsl.ImageStats(), name="num_of_voxels")
num_of_voxels.inputs.in_file = mask_file
num_of_voxels.inputs.op_string = "-V"

In [8]:
# workflow creation and node connection
random_effects_analysis = Workflow(name="random_effects_analysis",base_dir=output)

random_effects_analysis.connect(infosource,"contrast_list",datasink,"container")
random_effects_analysis.connect(infosource,"contrast_list",selectfiles,"contrast_list")
random_effects_analysis.connect(selectfiles,"copes",copemerge,"in_files")
random_effects_analysis.connect(selectfiles,"varcopes",varcopemerge,"in_files")
random_effects_analysis.connect(copemerge,"merged_file",random_eff_model,"cope_file")
random_effects_analysis.connect(varcopemerge,"merged_file",random_eff_model,"var_cope_file")
random_effects_analysis.connect(level2model,"design_mat",random_eff_model,"design_file")   
random_effects_analysis.connect(level2model,"design_con",random_eff_model,"t_con_file") 
random_effects_analysis.connect(level2model,"design_grp",random_eff_model,"cov_split_file") 
random_effects_analysis.connect(random_eff_model,"zstats",smooth_estimate,"zstat_file")
random_effects_analysis.connect(random_eff_model,"zstats",cluster_corr,"in_file") 
random_effects_analysis.connect(smooth_estimate,"dlh",cluster_corr,"dlh") 
random_effects_analysis.connect(num_of_voxels,("out_stat",pick_first),cluster_corr,"volume") 
random_effects_analysis.connect(copemerge,"merged_file",randomise,"in_file")
random_effects_analysis.connect(level2model,"design_con",randomise,"tcon")
random_effects_analysis.connect(level2model,"design_mat",randomise,"design_mat")

In [9]:
random_effects_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                          dotfilename= os.path.join(output,"random_effects_analysis_workflow.dot"))

# visualize workflow
img = Image.open(os.path.join(output,"random_effects_analysis_workflow.png"))
img.show()

231013-10:17:12,578 nipype.workflow INFO:
	 Generated workflow graph: /home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis_workflow.png (graph2use=flat, simple_form=True).


In [10]:
# connect to datasink
random_effects_analysis.connect(selectfiles,"copes",datasink,"copes")
random_effects_analysis.connect(selectfiles,"varcopes",datasink,"varcopes")
random_effects_analysis.connect(copemerge,"merged_file",datasink,"merge.@merged_cope_file")
random_effects_analysis.connect(varcopemerge,"merged_file",datasink,"merge.@merged_varcope_file")
random_effects_analysis.connect(level2model,"design_mat",datasink,"model.@design_mat")
random_effects_analysis.connect(random_eff_model,"copes",datasink,"random_effects.@copes")
random_effects_analysis.connect(random_eff_model,"var_copes",datasink,"random_effects.@var_copes")
random_effects_analysis.connect(random_eff_model,"weights",datasink,"random_effects.@weights")
random_effects_analysis.connect(random_eff_model,"zstats",datasink,"random_effects.@zstats")
random_effects_analysis.connect(random_eff_model,"res4d",datasink,"random_effects.@res4d")
random_effects_analysis.connect(random_eff_model,"tstats",datasink,"random_effects.@tstats")
random_effects_analysis.connect(cluster_corr,"threshold_file",datasink,"cluster.@z_cluster_corrected")
random_effects_analysis.connect(cluster_corr,"pval_file",datasink,"cluster.@pval_file")
random_effects_analysis.connect(cluster_corr,"localmax_txt_file",datasink,"cluster.@localmax_txt_file")
random_effects_analysis.connect(cluster_corr,"index_file",datasink,"cluster.@index_file")
random_effects_analysis.connect(randomise,"t_corrected_p_files",datasink,"non_parametric.@corr_p_values")

In [11]:
# run third level (RFX)
random_effects_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 1})

231013-10:17:12,698 nipype.workflow INFO:
	 Workflow random_effects_analysis settings: ['check', 'execution', 'logging', 'monitoring']
231013-10:17:12,799 nipype.workflow INFO:
	 Running in parallel.
231013-10:17:12,874 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 17 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:17:12,950 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.l2model" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/l2model".
231013-10:17:12,956 nipype.workflow INFO:
	 [Node] Executing "l2model" <nipype.interfaces.fsl.model.L2Model>
231013-10:17:12,961 nipype.workflow INFO:
	 [Node] Finished "l2model", elapsed time 0.000901s.
231013-10:17:14,878 nipype.workflow INFO:
	 [Job 0] Completed (random_effects_analysis.l2model).
231013-10:17:14,881 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 16 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:17:14,950 nipype.w

231013-10:17:36,954 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.selectfiles" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_10/selectfiles".
231013-10:17:36,958 nipype.workflow INFO:
	 [Node] Executing "selectfiles" <nipype.interfaces.io.SelectFiles>
231013-10:17:36,962 nipype.workflow INFO:
	 [Node] Finished "selectfiles", elapsed time 0.001933s.
231013-10:17:38,905 nipype.workflow INFO:
	 [Job 12] Completed (random_effects_analysis.selectfiles).
231013-10:17:38,908 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 26 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:17:38,949 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.selectfiles" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_11/selectfiles".
231013-10:17:38,951 nipype.workflow INFO:
	 [Node] Executing "selectfiles" <nipype.interfaces.io.SelectFiles>
231013-10:17:38,956 

231013-10:18:01,369 nipype.workflow INFO:
	 [Node] Finished "varcopemerge", elapsed time 0.385138s.
231013-10:18:02,931 nipype.workflow INFO:
	 [Job 24] Completed (random_effects_analysis.varcopemerge).
231013-10:18:02,937 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 30 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:18:02,980 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.copemerge" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_4/copemerge".
231013-10:18:02,989 nipype.workflow INFO:
	 [Node] Executing "copemerge" <nipype.interfaces.fsl.utils.Merge>
231013-10:18:03,407 nipype.workflow INFO:
	 [Node] Finished "copemerge", elapsed time 0.416388s.
231013-10:18:04,933 nipype.workflow INFO:
	 [Job 25] Completed (random_effects_analysis.copemerge).
231013-10:18:04,939 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 30 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.

231013-10:18:27,5 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.copemerge" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_10/copemerge".
231013-10:18:27,10 nipype.workflow INFO:
	 [Node] Executing "copemerge" <nipype.interfaces.fsl.utils.Merge>
231013-10:18:27,398 nipype.workflow INFO:
	 [Node] Finished "copemerge", elapsed time 0.384738s.
231013-10:18:28,961 nipype.workflow INFO:
	 [Job 37] Completed (random_effects_analysis.copemerge).
231013-10:18:28,968 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 30 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:18:29,12 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.varcopemerge" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_10/varcopemerge".
231013-10:18:29,18 nipype.workflow INFO:
	 [Node] Executing "varcopemerge" <nipype.interfaces.fsl.utils.Merge>
231013-10:18:29,404 nipype.wor

231013-10:19:19,69 nipype.workflow INFO:
	 [Node] Executing "randomise" <nipype.interfaces.fsl.model.Randomise>
231013-10:19:21,13 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 28 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * random_effects_analysis.randomise
231013-10:19:46,796 nipype.workflow INFO:
	 [Node] Finished "randomise", elapsed time 27.724601s.
231013-10:19:47,39 nipype.workflow INFO:
	 [Job 49] Completed (random_effects_analysis.randomise).
231013-10:19:47,47 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 28 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:19:47,112 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.random_eff_model" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_1/random_eff_model".
231013-10:19:47,119 nipype.workflow INFO:
	 [Node] Executing "random_eff_model" <nipype.

...

231013-10:26:39,499 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.random_eff_model" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_11/random_eff_model".
231013-10:26:39,506 nipype.workflow INFO:
	 [Node] Executing "random_eff_model" <nipype.interfaces.fsl.model.FLAMEO>
231013-10:26:40,814 nipype.workflow INFO:
	 [Node] Finished "random_eff_model", elapsed time 1.305012s.
231013-10:26:41,452 nipype.workflow INFO:
	 [Job 70] Completed (random_effects_analysis.random_eff_model).
231013-10:26:41,454 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 18 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:26:41,497 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.randomise" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_12/randomise".
231013-10:26:41,503 nipype.workflow INFO:
	 [Node] Executing "randomise" <nipype.interfaces.fsl.model.Rando

231013-10:28:51,628 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.smooth_estimate" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_4/smooth_estimate".
231013-10:28:51,633 nipype.workflow INFO:
	 [Node] Executing "smooth_estimate" <nipype.interfaces.fsl.model.SmoothEstimate>
231013-10:28:51,765 nipype.workflow INFO:
	 [Node] Finished "smooth_estimate", elapsed time 0.129909s.
231013-10:28:53,581 nipype.workflow INFO:
	 [Job 81] Completed (random_effects_analysis.smooth_estimate).
231013-10:28:53,589 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 15 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:28:53,631 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.smooth_estimate" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_5/smooth_estimate".
231013-10:28:53,636 nipype.workflow INFO:
	 [Node] Executing "smooth_estimate" <nipype.interfa

231013-10:29:15,670 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.cluster_corr" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_1/cluster_corr".
231013-10:29:15,678 nipype.workflow INFO:
	 [Node] Executing "cluster_corr" <nipype.interfaces.fsl.model.Cluster>
231013-10:29:15,851 nipype.workflow INFO:
	 [Node] Finished "cluster_corr", elapsed time 0.170855s.
231013-10:29:17,610 nipype.workflow INFO:
	 [Job 93] Completed (random_effects_analysis.cluster_corr).
231013-10:29:17,615 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 15 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:29:17,657 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.cluster_corr" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_2/cluster_corr".
231013-10:29:17,664 nipype.workflow INFO:
	 [Node] Executing "cluster_corr" <nipype.interfaces.fsl.model.Cluster>
231013-1

231013-10:29:39,719 nipype.workflow INFO:
	 [Node] Executing "cluster_corr" <nipype.interfaces.fsl.model.Cluster>
231013-10:29:39,887 nipype.workflow INFO:
	 [Node] Finished "cluster_corr", elapsed time 0.165387s.
231013-10:29:41,638 nipype.workflow INFO:
	 [Job 105] Completed (random_effects_analysis.cluster_corr).
231013-10:29:41,645 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 15 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:29:41,686 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.cluster_corr" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_14/cluster_corr".
231013-10:29:41,693 nipype.workflow INFO:
	 [Node] Executing "cluster_corr" <nipype.interfaces.fsl.model.Cluster>
231013-10:29:41,860 nipype.workflow INFO:
	 [Node] Finished "cluster_corr", elapsed time 0.16271s.
231013-10:29:43,641 nipype.workflow INFO:
	 [Job 106] Completed (random_effects_analysis.cluster_corr).
231013-10:2

231013-10:30:05,718 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.datasink" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_11/datasink".
231013-10:30:05,731 nipype.workflow INFO:
	 [Node] Executing "datasink" <nipype.interfaces.io.DataSink>
231013-10:30:05,782 nipype.workflow INFO:
	 [Node] Finished "datasink", elapsed time 0.049427s.
231013-10:30:07,667 nipype.workflow INFO:
	 [Job 118] Completed (random_effects_analysis.datasink).
231013-10:30:07,676 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231013-10:30:07,719 nipype.workflow INFO:
	 [Node] Setting-up "random_effects_analysis.datasink" in "/home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis/_contrast_list_12/datasink".
231013-10:30:07,731 nipype.workflow INFO:
	 [Node] Executing "datasink" <nipype.interfaces.io.DataSink>
231013-10:30:07,780 nipype.workflow INFO:
	 [Node]

In [12]:
print('Third level is done!')

Third level is done!


Cleanup - working folder was 'random_effects_analysis' but final files are in the numbered folders.  
Let's go ahead and delete the 'random_effects_analysis' folder.

In [13]:
!rm -r /home/anakin/Desktop/ds000113/output/3rd_level/random_effects_analysis

We will need to rename 3rd level output folders to a 2-digit format (i.e. 01,02,03...).   
For that, we will use a function similar to 1st level to rename folders - 'rename_numeric_folders'.  
Some of the folders have another subfolder in theme that we do not need, we will remove them for convenience using another function:  
'extract_files_from_subfolders'

In [15]:
def extract_files_from_subfolders(dir_path):
    """
    a function used for moving files within 3rd level output folders (contrast folders),
    for convenience of future path work.
    
    Input: dir_path [str]
    Output: a list of the final folders (paths)
    """
    # get a list of all subdirectories within the given directory
    sub_dirs = [d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]
    # iterate through each subdirectory
    for sub_dir in sub_dirs:
        sub_dir_path = os.path.join(dir_path, sub_dir)
        # get a list of subdirectories within this subdirectory
        sub_sub_dirs = [d for d in os.listdir(sub_dir_path) if os.path.isdir(os.path.join(sub_dir_path, d))]
        # if there are subdirectories within this subdirectory, extract files to the original subfolder
        if sub_sub_dirs:
            # iterate through sub-subdirectories
            for sub_sub_dir in sub_sub_dirs:
                sub_sub_dir_path = os.path.join(sub_dir_path, sub_sub_dir)
                # get a list of files in the sub-subdirectory
                files = [f for f in os.listdir(sub_sub_dir_path) if os.path.isfile(os.path.join(sub_sub_dir_path, f))]
                # move each file to the original subdirectory
                for file in files:
                    source_path = os.path.join(sub_sub_dir_path, file)
                    destination_path = os.path.join(sub_dir_path, file)
                    # check for filename conflicts and handle them by appending numbers
                    while os.path.exists(destination_path):
                        base, ext = os.path.splitext(destination_path)
                        destination_path = base + "_1" + ext
                    shutil.move(source_path, destination_path)
                # remove the empty sub-subdirectory
                os.rmdir(sub_sub_dir_path)
        # recursively check subdirectories within this subdirectory
        extract_files_from_subfolders(sub_dir_path)

def rename_numeric_folders(dir_path):
    """
    a function to rename 3rd level output from pipeline by formatting the folders' name
    to 2-digit and to start at 1. the final result will work given less than 100 contrasts
    and will look as follows: 01,02,03...N
    
    Input: dir_path [str]
    Output: None
    """
    # get a list of all number-named subdirectories (contrasts) within the given directory
    sub_dirs = [d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d)) and d.isdigit()]
    # descend sort 
    sub_dirs = [int(d)for d in sub_dirs]
    sub_dirs.sort(reverse=True)
    # make new numbers by adding 1 to start namins from 1 instead of 0
    numbersNew = [int(i)+1 for i in sub_dirs]
    # format to 2-digit
    formated = ["{:02d}".format(i) for i in numbersNew]
    # descend sort
    formated.sort(reverse=True)
    # iterate through each subdirectory in decsending order
    new_paths = []
    for i,sub_dir in enumerate(sub_dirs):
        # old path
        sub_dir_path = os.path.join(dir_path, str(sub_dir))
        new_path = os.path.join(dir_path, formated[i])
        # verify
        print('Changing',sub_dir_path)
        print('To',new_path)
        # rename folder
        os.rename(sub_dir_path, new_path)
        new_paths.append(new_path)
        
    return new_paths

In [16]:
# run rename_numeric_folders first, as it returns a path list of all contrasts
path_list = rename_numeric_folders('/home/anakin/Desktop/ds000113/output/3rd_level/')
# iterate over path list and flatten file tree in each folder
for path in path_list:
    extract_files_from_subfolders(path)

Changing /home/anakin/Desktop/ds000113/output/3rd_level/14
To /home/anakin/Desktop/ds000113/output/3rd_level/15
Changing /home/anakin/Desktop/ds000113/output/3rd_level/13
To /home/anakin/Desktop/ds000113/output/3rd_level/14
Changing /home/anakin/Desktop/ds000113/output/3rd_level/12
To /home/anakin/Desktop/ds000113/output/3rd_level/13
Changing /home/anakin/Desktop/ds000113/output/3rd_level/11
To /home/anakin/Desktop/ds000113/output/3rd_level/12
Changing /home/anakin/Desktop/ds000113/output/3rd_level/10
To /home/anakin/Desktop/ds000113/output/3rd_level/11
Changing /home/anakin/Desktop/ds000113/output/3rd_level/9
To /home/anakin/Desktop/ds000113/output/3rd_level/10
Changing /home/anakin/Desktop/ds000113/output/3rd_level/8
To /home/anakin/Desktop/ds000113/output/3rd_level/09
Changing /home/anakin/Desktop/ds000113/output/3rd_level/7
To /home/anakin/Desktop/ds000113/output/3rd_level/08
Changing /home/anakin/Desktop/ds000113/output/3rd_level/6
To /home/anakin/Desktop/ds000113/output/3rd_level